In [1]:
import requests as req
import re

In [2]:
from db import CATE_DB, DB

In [18]:
ENT = 'https://tuchong.com/explore/'
API = 'https://tuchong.com/rest/tag-categories/{}?page={}&count=20'

In [4]:
TAG_API = 'https://tuchong.com/rest/tags/{}/posts?page={}&count=20&order=weekly'

In [25]:
reCate = re.compile(r'href="https:\/\/tuchong\.com\/categories\/(.+)\/"')

In [13]:
res = req.get(ENT)
html = res.text

In [15]:
categories = reCate.findall(html)

In [36]:
for cat in categories:
    url = API.format(cat, 1)
    res = req.get(url)
    data = res.json()['data']
    
    CATE_DB.insert_many(data['tag_list'])
    
    pages = data['pages']
    for page in range(2, pages+1):
        url  = API.format(cat, page)
        res = req.get(url)
        data = res.json()['data']
        CATE_DB.insert_many(data['tag_list'])

In [3]:
tags = CATE_DB.find({})

In [7]:
for tag in tags:
    page = 1
    while True:
        url = TAG_API.format(tag['tag_name'], page)
        res = req.get(url)
        
        pl = res.json().get('postList')
        if not pl:
            break
        DB.insert_many(pl)
        page += 1
    

CursorNotFound: Cursor not found, cursor id: 53464674855

In [1]:
import pandas as pd

In [ ]:
df = pd.DataFrame(list(DB.find({})))